In [1]:
# 라이브러리 임포트
# etc
import os, sys
import glob
import csv
import cv2
import tqdm
from typing import Tuple, List, Dict
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# torch library
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
# torchvision library
import torchvision
from torchvision import transforms, models
import torch.optim as optim

In [ ]:
# Dataset

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

def get_label_from_filename(filename):
    age_str = filename.split('_')[2]
    age = int(age_str)
    if age <= 9:
        return '0'
    elif 10 <= age <= 19:
        return '1'
    elif 20 <= age <= 29:
        return '2'
    elif 30 <= age <= 39:
        return '3'
    elif 40 <= age <= 49:
        return '4'
    else:
      return '5'


train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 데이터 로딩
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Project/image_data/train', transform=train_transform)
val_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Project/image_data/valid', transform=val_transform)
test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Project/image_data/test', transform=val_transform)

# 배치 사이즈와 train:validation 비율 정의
batch_size = 8
# batch_size = 16

# 데이터로더 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# 클래스 정의
classes = ('0', '1', '2', '3', '4', '5')

In [ ]:
train_dataset

In [ ]:
# 이미지 데이터 시각화
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# 학습 이미지 얻기
dataiter = iter(train_loader)
images, labels = next(dataiter)
# 이미지 출력
imshow(torchvision.utils.make_grid(images))
# 라벨 프린트
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os
import numpy as np
from typing import Tuple, List
from torch import Tensor

# 커스텀 데이터셋 클래스
class CUSTOMDataset(Dataset):
    def __init__(self, root: str, transforms: transforms = None):
        self.root = root
        self.transforms = transforms
        self.images = []
        self.labels = []

        for folder in os.listdir(self.root):
            if folder == '.DS_Store':
                continue

            folder_path = os.path.join(self.root, folder)
            for filename in os.listdir(folder_path):
                self.images.append(os.path.join(folder_path, filename))

                # 파일명에서 라벨 부분 추출
                label_str = filename.split('_')[2] # 파일명에서 라벨 부분(예: '01') 추출
                label = int(label_str) // 10 if int(label_str) < 60 else 6
                self.labels.append(label)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_path = self.images[index]
        image = Image.open(image_path).convert('RGB')
        if self.transforms:
            image = self.transforms(image)

        label = self.labels[index]
        return image, label

In [ ]:
# 커스텀 데이터셋 & 로더
custom_dataset = CUSTOMDataset('/content/drive/MyDrive/Project/image_data/test', transforms = val_transform)
custom_loader = DataLoader(
    custom_dataset, batch_size=8, shuffle=False, num_workers=2)

In [ ]:
# 디바이스 체크 & 할당
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def train(epoch):
  train_loss = 0.0
  model.train()
  for i, data in enumerate(tqdm.tqdm(train_loader), 0):
      # 입력 데이터 가져오기 data: [inputs, labels]
      inputs, labels = data[0].to(device), data[1].to(device)

      # parameter gradients를 제로화
      optimizer.zero_grad()

      # 입력 이미지에 대한 출력 생성
      outputs = model(inputs)

      # 손실함수 계산 밎 업데이트
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()

  return train_loss

In [ ]:
def val():
  val_loss = 0.0
  val_accuracy = 0.0
  with torch.no_grad():
    # 모델 평가 모드 설정
    model.eval()
    for i, data in enumerate(tqdm.tqdm(val_loader), 0):
      # 입력 데이터 가져오기 data: [inputs, labels]
      inputs, labels = data[0].to(device), data[1].to(device)

      # 입력 이미지에 대한 출력 생성
      outputs = model(inputs)

      # 손실함수 계산
      loss = criterion(outputs, labels)
      val_loss += loss.item()

      # 예측 라벨
      _, predicted = torch.max(outputs, 1)

      # accuracy 계산
      val_accuracy += (predicted == labels).sum().item()

  return val_loss, val_accuracy

In [ ]:
def test(test_loader):
    correct = 0
    total = 0
    correct_class = {classname: 0 for classname in classes}
    total_class = {classname: 0 for classname in classes}
    model.eval()
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            # 입력 이미지에 대한 출력 생성
            outputs = model(inputs)
            # 예측 라벨
            _, predicted = torch.max(outputs.data, 1)
            # 전체 정확도 계산
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # 클래스 별 정확도 계산
            for label, prediction in zip(labels, predicted):
                if label == prediction:
                    correct_class[classes[label]] += 1
                total_class[classes[label]] += 1
    # 전체 정확도 출력
    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
    # 클래스 별 정확도 출력
    for classname, correct_count in correct_class.items():
        if total_class[classname] == 0:
          continue
        accuracy = 100 * float(correct_count) / total_class[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
# 학습 에포크 설정
train_epochs = 10
best_acc = 0.0
# 모델 저장 경로 정의
model_path = '/content/drive/MyDrive/Project/models/6class_resnet101.pth'
for epoch in range(train_epochs):
  # 학습 메소드 실행
  train_loss = train(epoch)
  print(f'[{epoch + 1}] loss: {train_loss / len(train_loader):.3f}')
  # 검증 메소드 실행
  val_loss, val_acc = val()
  vaild_acc = val_acc / (len(val_loader)*batch_size)
  print(f'[{epoch + 1}] loss: {val_loss / len(val_loader):.3f} acc: {vaild_acc:.3f}')
  # 정확도가 기존 베스트를 갱신할 경우 모델 저장
  if vaild_acc >= best_acc:
    best_acc = vaild_acc
    torch.save(model.state_dict(), model_path)
print('Finished Training')

In [ ]:
# 안됨
model_path = '/content/7class_resnet.pth'
# 모델 가중치 로드
model.load_state_dict(torch.load(model_path))

In [ ]:
# 테스트 메소드 실행
test(custom_loader)

In [ ]:
model_path = '/content/cifar_resnet.pth'
# 모델 가중치 로드
model.load_state_dict(torch.load(model_path))
# 테스트 메소드 실행
test(test_loader)